# Laboratorio 3: Programación Lineal Mixta (MA4702, otoño 2023)

Fecha: 17 de Mayo, 2023.<br/>
Profesor: Marcos Kiwi.<br/>
Auxiliares: Benjamín Jauregui y Cristián Palma.<br/>

## Preliminares

Este laboratorio consta de 3 etapas. Una tarea introductoria, una etapa presencial y otra no-presencial.
La nota global del laboratorio será 30 % (TI) + 40 % (TP) + 30 % (TNP), donde (TI) es la nota de la tarea
introductoria, (TP) la nota del trabajo presencial y (TNP) la nota del trabajo no presencial.

La tarea introductoria es de carácter estrictamente individual. La etapa presencial y no-presencial son de carácter grupal. 

## Laboratorio 3: Tarea Introductoria (TI)

## Instrucciones

* La respuesta a esta tarea consiste en completar este mismo archivo `.ipnyb` y entregarlo a través de Tareas en UCursos, antes de las **14:15 del miércoles 17 de mayo de 2023** (Se descontará 1.0 pts. por cada 15 minutos o fracción de atraso.)

* Realice esta primera tarea de manera **individual**. Bajo ningún motivo discuta sus respuestas, ni comparta o deje a disposición los archivos con sus compañeros. Se hará un estudio de similitudes entre los archivos resultantes para detectar posibles plagios.

<div class="alert alert-block alert-warning">
    Advertencia: Si en el archivo que entregue alguna de las celdas arroja un mensaje de error al ejecutar o el código entra en un loop infinito y no retorna con ninguna salida, Ud. deberá agregar una nota al respecto, explicando cual cree que es el problema. Si no lo hace, se le penalizará adicionalmente.               
</div>


## Datos personales

<div class="alert alert-block alert-success">
    Indique su nombre: Joaquín Márquez<br/>              
</div>


## Introducción

El objetivo de esta Tarea Introductoria es familiarizarse con formulaciones extendidas de problemas combinatoriales. En particular, del *Cutting Stock Problem (CSP)*.


Suponga que se tiene un inventario ilimitado de barras de largo $W\in\mathbb{N}$. Suponga además que para $i\in [m]$ hay una demanda de $d_i\in\mathbb{N}$ barras de largo $w_i$. Sin pérdida de generalidad, en lo que sigue asumimos que $w_1>w_2>...>w_m$.

El CSP consiste en satisfacer la demanda usando el mínimo número de barras del inventario. Se propone formular CSP como un problema de minimización de flujo en el digrafo $G=(V,A)$ donde $V=\{0,1,...,W\}$ y 
$$
A = \{ (u,v)\in V^2 \mid v-u\in\{w_1,...,w_m\}\} \cup \{(u,v)\in V^2 \mid v=u+1\}.
$$
**Nota:** Por simplicidad, asuma que $w_i\neq 1$ para todo $i\in [m]$.

## Ejercicio 1

1a) Basándose en que un $(0,W)$-dicamino en $G$ representa un patrón de corte de una barra de largo $W$ y que tal dicamino está determinado por un $(0,W)$-flujo unitario en $G$, formule el CSP como un PLE.

<div class="alert alert-block alert-warning">
    Respuesta Ejercicio 1a
</div>
Veamos que, en efecto, todo $(0,W)$-dicamino en $G$ representa un patrón de corte en una barra de largo $W$. ¿Cómo logramos esto? Pues considerando que si tenemos un $(0,W)$-dicamino $C=0u_1\cdots u_{n-1}W$, entonces podemos notar que, dado que $w_i\not=1$, entonces todos las aristas $u_iu_{i+1}$ que son de la forma tal que $u_{i+1}-u_i = w_i\in \{w_k\}_{k=1}^m$, representan un corte de tamaño $w_i$, y cuando $u_{i+1}-u_i=1$, entonces tendríamos que no son usados para ningún elemento del pedido.
    
Podríamos considerar entonces la superposición de tantos caminos como queramos, estos dicaminos tienen flujo $1$ en ese sentido (ya que sería ir recorriendo de igual "ritmo" cada nodo que se recorre). Podríamos sumar cuantos caminos queramos y en estos se cumple que es un flujo, ya que se puede descomponer por caminos de valor constante en cada caso (en este caso es mejor aún, porque ese valor es $1$).

Perfecto entonces, podemos escribir esto como un flujo, luego debemos agregarle la restricción de que la cantidad de elementos que representan un corte de tamaño $w_i$ sean iguales (o mayores) a la demanda. Sin pérdida de generalidad se pueden tomar como igualdad, ya que si tenemos una desigualdad estricta en algún caso $d_j$, podemos descomponer en dicaminos el flujo que vamos a optimzar, existirá algún dicamino con almenos un $x_{i,i+w_j}$ en él. Podemos reemplazarlo por una sucesión de aristas $x_{i,i+1}x_{i+1,i+2},\cdots, x_{i+w_j-1, i+w_j}$. Se mantiene en cualquier caso el flujo total ya que el número de dicaminos se mantiene constante. Así obtenemos el siguiente problema:
\begin{align*}
\min &\sum_{i=0}^{W} x_{i,W}\\
\text{s.a. } & x \text{ flujo}\\
& \sum_{i=0}^{W} x_{i,i+w_j} &=& d_j & (\forall j\in[m])\\
& x_e &\geq& 0 & (\forall e \in A)
\end{align*}
Recordando que la característica esencial del flujo es que las cosas no se crean ni destruyen, salvo en los extremos, la formulación se vuelve:
\begin{align*}
\min &\sum_{i=0}^{W} x_{i,W}\\
\text{s.a. } & \sum_{j=0}^W x_{j,i} &=& \sum_{j=0}^W x_{i,j} & (\forall i \in [W-1]\backslash\{0\})\\
& \sum_{i=0}^{W} x_{i,i+w_j} &=& d_j & (\forall j\in[m])\\
& x_e &\geq& 0 & (\forall e \in A)\\
& x_e &\in& \mathbb{N} & (\forall e\in A)
\end{align*}
Donde por temas de nomenclatura, cuando $i+w_j>W$, vamos a considerarlo como una suma vacía (es decir, se cuenta cada sumatoria allí hasta que el $i+w_j$ no sea válido).

1b) Complete (donde se indica) el código de la siguiente celda de forma que la función `csp1(dtsInst)` optimice y retorne una formulación en `JuMP` de $(CSP)$ para la instancia que se especifica a través del arreglo `dstInst` (ver comentarios en el código que indican como se codifica la instancia). 

In [13]:
using JuMP, Gurobi;
const GUROBI_ENV = Gurobi.Env();

function csp1(dtsInst)
  # Función que crea el modelo a resolver.

  mdl = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV), "OutputFlag" => 0));

  W = dtsInst[1];              # Largo barras en inventario.
  m = (size(dtsInst,2)-1) ÷ 2  # No. de largos distintos de barras a cortar.
  wgh = dtsInst[2:m+1];        # Largos de las m barras que se pueden cortar (wgh[1] > wgh[2] > ... > wgh[m])
  d = dtsInst[m+2:2m+1];       # d[i] = demanda por barras de largo wgh[i]
    
  # 
  # Crear modelo
  #
  # Comenzar a agregar código a continuación.
  @variable(mdl, z[i=1:W,j=1:W],Int)
  # Esto da que las variables que trabajaremos sean enteras.
  @objective(mdl, Min, sum(z[i,W] for i=1:W))
  # Ahí tenemos lista la función objetivo
  @constraint(mdl,[j=2:W-1], sum(z[j,i] for i=1:W) == sum(z[i,j] for i=1:W)) # Definición flujo
  @constraint(mdl,[j=1:m], sum(z[i,i+wgh[j]] for i=1:(W-wgh[j])) == d[j]) # Cumple la demanda de forma exacta
  @constraint(mdl,[j=1:W-1], z[j,j+1] >= 0) # Restricción sobre los vecinos inmediatos
  @constraint(mdl,[j=1:m, i=1:W-wgh[j]], z[i,i+wgh[j]] >= 0) # Restricción sobre los representantes de las barras w_j
  for i=1:W
      for j=1:W
          if (i-j !in wgh) && (i-j != 1)
              @constraint(mdl, z[i,j] == 0) # No tomará el resto de posibles caminos.
          end
      end
  end
  # Terminar de agregar código.
  @time optimize!(mdl)
    
  return mdl
end

Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-09


LoadError: syntax: missing comma or ) in argument list

No sé porque no compila, creo que escribí algo mal en el ciclo for. Pero la idea era fijar todas las posibles "aristas restantes" con capacidad 0 por así decirlo.

Ejecute la siguiente celda para resolver las instancias que se proveen.

In [6]:
using DelimitedFiles;

dirInst = "dirTI";      # Directorio con archivos de instancias.

rslt1 = [];
for fn in readdir(dirInst)
  dtsInst = readdlm(joinpath(dirInst,fn), ',', Int64, '\n')

  println( repeat('=', 60) );
  println( "Instancia ", fn );
  println( repeat('=', 60) );
  mdl = csp1(dtsInst)
  if termination_status(mdl) == MOI.OPTIMAL
    obj = objective_value(mdl);
  else
    obj = Inf
  end

  # Guarda estadísticas para análisis posterior.
  push!(rslt1, obj);
end

# Imprime resumen de resultados.
println( "" ); println( "RESUMEN:")
println( "Instancia | Valor " );
for i in 1:size(rslt1,1)
  print( "   ", i, "         ")
  println( rslt1[i] ) 
end

Instancia csp-i001.txt
  0.072386 seconds (1.30 k allocations: 111.891 KiB)
Instancia csp-i002.txt
  0.006536 seconds (1.13 k allocations: 98.484 KiB)
Instancia csp-i003.txt
  0.010974 seconds (929 allocations: 82.422 KiB)
Instancia csp-i004.txt
  1.382409 seconds (2.02 M allocations: 207.114 MiB)
Instancia csp-i005.txt
  0.051340 seconds (83.26 k allocations: 8.651 MiB)
Instancia csp-i006.txt
  4.821919 seconds (8.04 M allocations: 841.280 MiB, 12.34% gc time)
Instancia csp-i007.txt
 32.684542 seconds (32.10 M allocations: 3.297 GiB, 14.06% gc time)

RESUMEN:
Instancia | Valor 
   1         0.0
   2         0.0
   3         0.0
   4         0.0
   5         0.0
   6         0.0
   7         0.0


## Ejercicio 2

Se definen: 

1. $A_1=\{(u,u+1)\in A \mid u < w_m\}$. 
2. $A_2=\cup_{j\in [m]} A_2(j)$ donde $A_2(j)=\{(v,v+w_j)\in A \mid v\neq 0, \forall i\in [j], (v-w_{i},v)\not\in A\}$ para todo $j\in [m]$.
3. $A_3=\cup_{j\in [m]} A_3(j)$ donde $A_3(j)=\{(v,v+w_j)\in A \mid (v-kw_j,v-(k-1)w_j)\in A, \forall k\in [d]\}$.

Sea $A'=A\setminus (A_1\cup A_2\cup A_3)$ y $(CSP')$ su formulación de $(CSP)$ pero considerando ahora $A'$ en reemplazo de $A$ (equivalentemente, $(CSP')$ es la formulación análoga a de $(CSP)$ de la parte anterior pero considerando el grafo $G'=(V,A')$ en vez de $G=(V,A)$). Sea $z'$ el valor óptimo de $(CSP')$.

Pruebe que $z=z'$.


<div class="alert alert-block alert-warning">
    Respuesta Ejercicio 2:  
    
    
    
    
    
</div>


La idea en este ejercicio es reducir la cantidad de simetrías del problema, ya que si tenemos una solución, en cada camino podemos armar, por como está creado el grafo, un reordenamiento de dichas aristas y seguiría siendo solución, esto da un gran problema para cuando se optimiza el caso. $A_1$ lo que hace es decir que la primera arista sí o sí debe representar una barra útil (dice menor a $w_m$ porque es la menor, entonces quitamos todas las aristas que solo se podían formar concatenando aristas que representaban el no utilizar una parte de la barra).

El $A_2$ representa que las aristas estén en orden según el largo que utilizarán, en este caso es en orden creciente (si la primera es la más pequeña, la siguiente debe ser algo más larga, de la contrario ocurre la restricción que dice ahí de que se tomó una arista menor o igual después) Nótese que esta sólo excluye las primeras, la $A_3$ completa el trabajo extendiendo esta noción a las siguientes. con $k$ y $k-1$.

Esto solo impone una noción de "ordenamiento" en los flujos que se van a optimizar, eliminando una simetría (de forma parcial por excluir el $A_3$ para este desarrollo). Por esto, ya que esto da un subconjunto pero como es reordenamiento la solución $z$ se puede transformar en $z'$, tenemos que deben ser iguales.